In [36]:
from datacube import Datacube
import fiona
from datacube.utils.geometry import CRS, Geometry
from shapely import geometry
from rasterio import features
import pandas as pd

In [3]:
def shape_list(shapefile):
    with fiona.open(shapefile) as allshapes:
        for shape in allshapes:
            yield shape

In [20]:
shapefile = './shapefile/IBRA_Regions/IBRA_MERGED_coastal.shp'
query_time = ('1987-01-01', '2021-01-01')
dc = Datacube()

In [144]:
for shape in shape_list(shapefile):
    region_geom = geometry.shape(shape['geometry'])
    query_poly = Geometry(region_geom.convex_hull, crs=CRS('EPSG:3577'))
    datasets = dc.find_datasets(product='mangrove_cover', time=query_time, geopolygon=query_poly, measurements=['cover'])
    results = pd.DataFrame()
    for d in datasets:
        data = dc.load(product='mangrove_cover', datasets=[d], measurements=['cover'])
        geobox = data.geobox 
        mask = features.geometry_mask(
            region_geom,
            out_shape=geobox.shape,
            transform=geobox.affine,
            all_touched=True,
            invert=False,
        )
        year = list(data.time.data)
        area = [0] * 3
        for i in range(1, 4):
            area[i-1] = data.cover.where((data.cover==i) & mask).count().data
        results = results.append(pd.DataFrame(data=[year+area], columns=['time', 'woodland', 'open_forest', 'closed_forest']), ignore_index=True)
    results = results.groupby('time').sum()
    (results * (25 ** 2)).to_csv(shape['properties'].get('REG_NAME_7')+'.csv')